In [ ]:
import os

import albumentations as A
import cv2
import matplotlib.pyplot as plt
# import numpy as np
import torch
from torch.utils.data import DataLoader

# from prepare_data import prepare_data
from segmentation.datasets import SegmentationDataset

%load_ext autoreload
%autoreload 2

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
DATA_DIR = "./data/CamVid/preproc/"
CATEGORIES = [
    "background",
    "sky", 
    "building", 
    "pole", 
    "road", 
    "pavement", 
    "tree", 
    "signsymbol", 
    "fence", 
    "car", 
    "pedestrian", 
    "bicyclist", 
    "unlabelled"
]
CATEGORY_IDS = range(len(CATEGORIES))
NUM_WORKERS = 16

In [ ]:
images_train_dir = os.path.join(DATA_DIR, "images", "train")
masks_train_dir = os.path.join(DATA_DIR, "masks", "train")

images_val_dir = os.path.join(DATA_DIR, "images", "val")
masks_val_dir = os.path.join(DATA_DIR, "masks", "val")

images_test_dir = os.path.join(DATA_DIR, "images", "test")
masks_test_dir = os.path.join(DATA_DIR, "masks", "test")

In [ ]:
def get_train_augments(size=512):
    transforms = [

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
        
        A.LongestMaxSize(size),
        A.PadIfNeeded(size, size, border_mode=cv2.BORDER_CONSTANT),

#         A.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
#         A.RandomCrop(height=320, width=320, always_apply=True),

        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.RandomContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return A.Compose(transforms)


def get_val_augments(size=512):
    transforms = [
        A.LongestMaxSize(size),
        A.PadIfNeeded(size, size, border_mode=cv2.BORDER_CONSTANT),
    ]
    return A.Compose(transforms)

def to_float32(x, **kwargs):
    return x.transpose(2, 0, 1).astype("float32")

def to_int64(x, **kwargs):
    return x.astype("int64")

def get_preprocess(preprocessing_fn):
    transforms = [
        A.Lambda(image=preprocessing_fn),
#         A.Lambda(image=to_float32, mask=to_float32),
        A.Lambda(image=to_float32, mask=to_int64),
    ]
    return A.Compose(transforms)

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
ENCODER_NAME = "resnet34"
ENCODER_DEPTH = 5
ENCODER_WEIGHTS = "imagenet"
ENCODER_OUTPUT_STRIDE = 16
DECODER_CHANNELS = 256
DECODER_ATROUS_RATES = (12, 24, 36)
IN_CHANNELS = 3
ACTIVATION = None # could be None for logits or "softmax2d" for multicalss segmentation
UPSAMPLING = 4
AUX_PARAMS = None
DEVICE = "cuda"
BATCH_SIZE = 8

# create segmentation model with pretrained encoder
model = smp.DeepLabV3Plus(
    encoder_name=ENCODER_NAME,
    encoder_depth=ENCODER_DEPTH,
    encoder_weights=ENCODER_WEIGHTS,
    encoder_output_stride=ENCODER_OUTPUT_STRIDE,
    decoder_channels=DECODER_CHANNELS,
    decoder_atrous_rates=DECODER_ATROUS_RATES,
    in_channels=IN_CHANNELS,
    classes=len(CATEGORY_IDS),
    activation=ACTIVATION,
    upsampling=UPSAMPLING,
    aux_params=AUX_PARAMS,
)

In [ ]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER_NAME, ENCODER_WEIGHTS)

train_dataset = SegmentationDataset(
    images_dir=images_train_dir,
    masks_dir=masks_train_dir,
    category_ids=CATEGORY_IDS,
    augments=get_train_augments(),
    preprocess=get_preprocess(preprocessing_fn),
)
val_dataset = SegmentationDataset(
    images_dir=images_val_dir,
    masks_dir=masks_val_dir,
    category_ids=CATEGORY_IDS,
    augments=get_val_augments(),
    preprocess=get_preprocess(preprocessing_fn),
)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS
)

In [ ]:
from pytorch_toolbelt import losses

In [ ]:
loss = losses.DiceLoss(mode="multiclass")
loss.__name__ = "DiceLoss"
# cross-entropy incosistenza tra il formato che vuole e quello della previsione
# metrics = [
#     smp.utils.metrics.IoU(threshold=0.5),
# ]
metrics = []

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
max_score = 0

for i in range(0, 40):
    
    print("\nEpoch: {}".format(i))
    train_logs = train_epoch.run(train_loader)
#     valid_logs = valid_epoch.run(val_loader)
    
#     # do something (save model, change lr, etc.)
#     if max_score < valid_logs["iou_score"]:
#         max_score = valid_logs["iou_score"]
#         torch.save(model, "./best_model.pth")
#         print("Model saved!")
        
    if i == 25:
        optimizer.param_groups[0]["lr"] = 1e-5
        print("Decrease decoder learning rate to 1e-5!")

In [ ]:
import pytorch_lightning as pl
from torch.optim import SGD

from segmentation.optim import PolyLR


class SegmentationModule(pl.LightningModule):
    def __init__(
        self,
        model,
        loss,
        lr=0.007,
        momentum=0.9,
        weight_decay=0.00004,
        max_step=30000,
        power=0.9,
    ):
        super().__init__()
        self.model = model
        self.loss = loss
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.max_step = max_step
        self.power = power

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        return loss

    def configure_optimizers(self):
        optimizer = SGD(
            self.parameters(),
            lr=self.lr,
            momentum=self.momentum,
            weight_decay=self.weight_decay,
        )
        scheduler = PolyLR(optimizer, self.max_step, power=self.power)
        return [optimizer], [scheduler]

In [ ]:
pl.seed_everything(42)
trainer = pl.Trainer(gpus=1)